## 1.0. Preliminaries

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.units import UnitsWarning
from astropy.coordinates import SkyCoord
import astropy.units as u
import warnings
from lsst.rsp import get_tap_service, retrieve_query

import healpy as hp
import math
import gc

In [16]:
warnings.simplefilter("ignore", category=UnitsWarning)
pd.set_option('display.max_rows', 200)

In [17]:
%matplotlib inline

## 2.0. Query TAP server

The <a href="dp0-2.lsst.io">DP0.2 Documentation</a> contains a <a href="https://dp0-2.lsst.io/data-products-dp0-2/index.html#catalogs">list of all DP0.2 catalogs</a>, and also a link to the <a href="https://dm.lsst.org/sdm_schemas/browser/dp02.html">DP0.2 Schema Browser</a> where users can read about the available tables and their contents.

Alternatively, the Portal Aspect of the Rubin Science Platform can be used to browse catalog data.

Below, we show how to browse catalog data from a Notebook using the TAP service.

To access tables, we will use the TAP service in a similar manner to what we showed in the [Intro to DP0 notebook](https://github.com/rubin-dp0/tutorial-notebooks/blob/main/01_Intro_to_DP0_Notebooks.ipynb), and explored further in the [TAP tutorial notebook](https://github.com/rubin-dp0/tutorial-notebooks/blob/main/02_Intermediate_TAP_Query.ipynb). See those notebooks for more details.

In [18]:
service = get_tap_service("tap")

### 2.1 Download all SNe from TruthSummary table.

In [5]:
%%time
# This cell takes about 1 minute...

query = """SELECT * FROM dp02_dc2_catalogs.MatchesTruth WHERE truth_type=3"""

print(query)

results = service.search(query)
df_mt3 = results.to_table().to_pandas()
del results

df_mt3

SELECT * FROM dp02_dc2_catalogs.MatchesTruth WHERE truth_type=3
CPU times: user 8.88 s, sys: 340 ms, total: 9.22 s
Wall time: 1min


,id,id_truth_type,match_candidate,match_chisq,match_count,match_n_chisq_finite,match_objectId,truth_type
0,MS_10444_2245,MS_10444_2245_3,False,NaN,0,0,<NA>,3
1,MS_10444_4105,MS_10444_4105_3,False,NaN,0,0,<NA>,3
2,MS_10444_4126,MS_10444_4126_3,False,NaN,0,0,<NA>,3
3,MS_10444_4152,MS_10444_4152_3,False,NaN,0,0,<NA>,3
4,MS_10444_4197,MS_10444_4197_3,False,NaN,0,0,<NA>,3
...,...,...,...,...,...,...,...,...
418180,MS_8921_892,MS_8921_892_3,False,NaN,0,0,<NA>,3
418181,MS_8921_902,MS_8921_902_3,False,NaN,0,0,<NA>,3
418182,MS_8921_919,MS_8921_919_3,False,NaN,0,0,<NA>,3
418183,MS_8921_947,MS_8921_947_3,False,NaN,0,0,<NA>,3


Note that this is the same number of entries that Akshita found.

### 2.2 Download all SNe from TruthSummary table within an RA/DEC box of within a box 62.0<ra<62.5 and -37.5<dec<-37.0.

(We restrict the box size to save time for this quick analysis.)

In [6]:
%%time
# This cell takes about 1.5 minutes...

query = """SELECT * FROM dp02_dc2_catalogs.TruthSummary 
           WHERE 
           truth_type=3  
           AND 
           ra BETWEEN 59.0 AND 60.3
           AND
           dec BETWEEN -37.4 AND -36.1
        """

print(query)

results = service.search(query)
df_ts3 = results.to_table().to_pandas()
del results

df_ts3

SELECT * FROM dp02_dc2_catalogs.TruthSummary 
           WHERE 
           truth_type=3  
           AND 
           ra BETWEEN 59.0 AND 60.3
           AND
           dec BETWEEN -37.4 AND -36.1
        
CPU times: user 187 ms, sys: 9.61 ms, total: 196 ms
Wall time: 1min 41s


,cosmodc2_hp,cosmodc2_id,dec,flux_g,flux_g_noMW,flux_i,flux_i_noMW,flux_r,flux_r_noMW,flux_u,...,flux_z_noMW,host_galaxy,id,id_truth_type,is_pointsource,is_variable,mag_r,ra,redshift,truth_type
0,-1,-1,-37.156152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7877165718,MS_9813_4175,MS_9813_4175_3,1,1,NaN,59.177446,0.971620,3
1,-1,-1,-37.065010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7875119162,MS_9813_208,MS_9813_208_3,1,1,NaN,59.372651,0.312840,3
2,-1,-1,-37.172915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7875154553,MS_9813_276,MS_9813_276_3,1,1,NaN,59.262496,0.337525,3
3,-1,-1,-37.159203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7875246520,MS_9813_471,MS_9813_471_3,1,1,NaN,59.360019,0.403027,3
4,-1,-1,-37.138482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7875247123,MS_9813_496,MS_9813_496_3,1,1,NaN,59.291531,0.424815,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1793,-1,-1,-36.308620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7877156397,MS_9813_4086,MS_9813_4086_3,1,1,NaN,59.200872,0.965715,3
1794,-1,-1,-36.499756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7877164930,MS_9813_4088,MS_9813_4088_3,1,1,NaN,59.274158,0.969897,3
1795,-1,-1,-36.431403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7877152431,MS_9813_4163,MS_9813_4163_3,1,1,NaN,59.120754,0.969590,3
1796,-1,-1,-36.491597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7877161076,MS_9813_4209,MS_9813_4209_3,1,1,NaN,59.090989,0.972668,3


### 2.3 Download all SNe from DiaObject table within an RA/DEC box of within a box 62.0<ra<62.5 and -37.5<dec<-37.0.

(We restrict the box size to save time for this quick analysis.)

In [7]:
%%time
# This cell takes about half a minute...

query = """SELECT ra, decl, diaObjectid, rPSFluxMax FROM dp02_dc2_catalogs.DiaObject
            WHERE 
            ra BETWEEN 59.0 AND 60.3
           AND
           decl BETWEEN -37.4 AND -36.1
        """

print(query)

results = service.search(query)
df_do = results.to_table().to_pandas()
del results

df_do

SELECT ra, decl, diaObjectid, rPSFluxMax FROM dp02_dc2_catalogs.DiaObject
            WHERE 
            ra BETWEEN 59.0 AND 60.3
           AND
           decl BETWEEN -37.4 AND -36.1
        
CPU times: user 2.33 s, sys: 57.9 ms, total: 2.39 s
Wall time: 33.4 s


,ra,decl,diaObjectid,rPSFluxMax
0,60.091447,-37.219288,1650797961849869133,1255.228763
1,60.059480,-37.073130,1650797961849869206,-941.879211
2,60.089940,-37.223306,1650797961849867580,NaN
3,60.057494,-37.066646,1650797961849869144,1167.086997
4,60.071346,-37.109419,1650797961849869212,1175.738682
...,...,...,...,...
179060,59.339568,-36.471127,1651000271989381699,NaN
179061,59.340104,-36.481942,1651000271989375868,1542.807108
179062,59.339734,-36.514002,1651000271989382270,NaN
179063,59.340297,-36.482034,1651000271989375258,2129.629770


## 3.0 Matching the DiaObject table with the TruthSummary table

Unfortunately, at the current time, there is no equivalent of the **MatchesTruth** table for the Dia tables.  The **MatchesTruth** table only has matches to the **Objects** table.  (The `match_objectid` column in the **MatchesTruth** table holds the `objectid` from the corresponding entry in the **Objects** table.)

Until there is a equivalent match id for Dia objects (or Dia sources), we must do the match ourselves.  An imperfect -- but still pretty good - means of matching is to match healpixel values for the RAs/DECs of the entries in the Dia table and the **TruthSummary** table.

In the following, we will do this for a subsamples of the **DiaObjects** and the **TruthSummary** tables we downloaded above (i.e., for a box 62.0<ra<62.5 and -37.5<dec<-37.0).

First, we define a couple of modules that make use of healpy.  These were copied from <a href="https://github.com/DouglasLeeTucker/DECam_PGCM/blob/master/python/healpixTools.py">this link</a>:

In [8]:
# Healpix tools.
#  From Sahar Allam (2016.07.08)

##################################
def radec2thetaphi(ra, dec):
    import numpy as np
    return (90-dec)*np.pi/180., ra*np.pi/180.

##################################
#DESDM uses nside=128, nest=True
#Alex Drlica Wagner's healpixelated Gaia DR2 on des40 uses nside=32, nest=False
def getipix(nside,ra,dec,nest=True):
    import healpy as hp
    theta, phi = radec2thetaphi(ra, dec)
    ipix = hp.pixelfunc.ang2pix(nside, theta, phi, nest)
    return ipix

##################################


To do a suitable match, we want to chose a healpixel area that is pretty small.  A healpixel of  `NSIDES` = 2^17 seems not unreasonable:

In [9]:
nside=2**17
# Area in sq arcsec:
area = 3600.*3600.*hp.pixelfunc.nside2pixarea(nside, degrees=True)
# A rough measure of length in arcsec: 
length = math.sqrt(area)
print(nside, area, length)

131072 2.593337450489216 1.6103842555394088


Let's add an "`NSIDES`=2^17" healpixel column to our **TruthSummary** DataFrame:

In [10]:
df_ts3.loc[:,'HPX_2_17'] = getipix(nside, df_ts3.loc[:,'ra'].values, df_ts3.loc[:,'dec'].values)
df_ts3

,cosmodc2_hp,cosmodc2_id,dec,flux_g,flux_g_noMW,flux_i,flux_i_noMW,flux_r,flux_r_noMW,flux_u,...,host_galaxy,id,id_truth_type,is_pointsource,is_variable,mag_r,ra,redshift,truth_type,HPX_2_17
0,-1,-1,-37.156152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7877165718,MS_9813_4175,MS_9813_4175_3,1,1,NaN,59.177446,0.971620,3,144760794568
1,-1,-1,-37.065010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7875119162,MS_9813_208,MS_9813_208_3,1,1,NaN,59.372651,0.312840,3,144760942985
2,-1,-1,-37.172915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7875154553,MS_9813_276,MS_9813_276_3,1,1,NaN,59.262496,0.337525,3,144760742779
3,-1,-1,-37.159203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7875246520,MS_9813_471,MS_9813_471_3,1,1,NaN,59.360019,0.403027,3,144760727518
4,-1,-1,-37.138482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7875247123,MS_9813_496,MS_9813_496_3,1,1,NaN,59.291531,0.424815,3,144760747957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1793,-1,-1,-36.308620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7877156397,MS_9813_4086,MS_9813_4086_3,1,1,NaN,59.200872,0.965715,3,144768218722
1794,-1,-1,-36.499756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7877164930,MS_9813_4088,MS_9813_4088_3,1,1,NaN,59.274158,0.969897,3,144768006221
1795,-1,-1,-36.431403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7877152431,MS_9813_4163,MS_9813_4163_3,1,1,NaN,59.120754,0.969590,3,144768145476
1796,-1,-1,-36.491597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7877161076,MS_9813_4209,MS_9813_4209_3,1,1,NaN,59.090989,0.972668,3,144767417029


Likewise, let's add an "`NSIDES`=2^17" healpixel column to our **DiaObjects** DataFrame:

In [11]:
df_do.loc[:,'HPX_2_17'] = getipix(nside, df_do.loc[:,'ra'].values, df_do.loc[:,'decl'].values)
df_do

,ra,decl,diaObjectid,rPSFluxMax,HPX_2_17
0,60.091447,-37.219288,1650797961849869133,1255.228763,144759318801
1,60.059480,-37.073130,1650797961849869206,-941.879211,144759891582
2,60.089940,-37.223306,1650797961849867580,NaN,144759317235
3,60.057494,-37.066646,1650797961849869144,1167.086997,144759893272
4,60.071346,-37.109419,1650797961849869212,1175.738682,144759882356
...,...,...,...,...,...
179060,59.339568,-36.471127,1651000271989381699,NaN,144768047497
179061,59.340104,-36.481942,1651000271989375868,1542.807108,144768045711
179062,59.339734,-36.514002,1651000271989382270,NaN,144767691198
179063,59.340297,-36.482034,1651000271989375258,2129.629770,144768045709


Now, let's match both DataFrames using an inner join on their `HPX_2_17` columns:

In [12]:
df_merged = pd.merge(df_ts3, df_do, how='inner', on=['HPX_2_17','HPX_2_17'])
df_merged

,cosmodc2_hp,cosmodc2_id,dec,flux_g,flux_g_noMW,flux_i,flux_i_noMW,flux_r,flux_r_noMW,flux_u,...,is_variable,mag_r,ra_x,redshift,truth_type,HPX_2_17,ra_y,decl,diaObjectid,rPSFluxMax
0,-1,-1,-37.065010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,NaN,59.372651,0.312840,3,144760942985,59.372613,-37.064935,1650331768919689263,-764.852317
1,-1,-1,-37.065010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,NaN,59.372651,0.312840,3,144760942985,59.372345,-37.065158,1650331768919689280,NaN
2,-1,-1,-37.065010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,NaN,59.372651,0.312840,3,144760942985,59.372747,-37.065108,1650331768919688281,NaN
3,-1,-1,-37.065010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,NaN,59.372651,0.312840,3,144760942985,59.372747,-37.065108,1650815554035909867,NaN
4,-1,-1,-37.065010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,NaN,59.372651,0.312840,3,144760942985,59.372345,-37.065158,1650815554035911127,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,-1,-1,-36.445867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,NaN,59.284046,0.961764,3,144768075437,59.283777,-36.446020,1650516486873154735,581.366382
502,-1,-1,-36.203745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,NaN,59.267261,0.911343,3,144769814886,59.267247,-36.203750,1650578059524310554,NaN
503,-1,-1,-36.306855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,NaN,59.302832,0.916809,3,144768202079,59.302874,-36.306842,1650578059524310558,NaN
504,-1,-1,-36.503908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,NaN,59.249757,0.966075,3,144768025601,59.249813,-36.503884,1650516486873157464,NaN


In [13]:
print(len(df_merged))

506


In [14]:
df_merged.to_csv('df_merged.csv')

How do the RA's/DEC's from the two merged tables compare?

In [ ]:
df_merged['diff_ra'] = 3600.*(df_merged['ra_x'] - df_merged['ra_y'])
df_merged['diff_dec'] = 3600.*(df_merged['dec'] - df_merged['decl'])

In [ ]:
df_merged.hist('diff_ra')

In [ ]:
df_merged.hist('diff_dec')

Not too bad.  Contamination is probably relatively low, and purity is probably relatively high.  For better results, an improved sky matching algorithm can be employed.

In [ ]:
df_merged.columns

In [ ]:
df_merged.plot('redshift', 'rPSFluxMax', kind='scatter')

In [ ]:
df_merged.redshift.size

In [ ]:
df_merged.to_csv('matched.out')

## 4.0 Try the same thing, but by matching using KDtrees

See https://docs.astropy.org/en/stable/coordinates/matchsep.html?highlight=matching#matching-catalogs

In [ ]:
c_ts3 = SkyCoord(ra=df_ts3.loc[:,'ra'].values*u.degree, dec=df_ts3.loc[:,'dec'].values*u.degree)

In [ ]:
c_do = SkyCoord(ra=df_do.loc[:,'ra'].values*u.degree, dec=df_do.loc[:,'decl'].values*u.degree)

In [ ]:
idx, d2d, d3d = c_ts3.match_to_catalog_sky(c_do)

In [ ]:
idx

In [ ]:
c_do[idx]

In [ ]:
df_do.loc[idx]

In [ ]:
df_do_new = df_do.loc[idx].copy(deep=True)

In [ ]:
df_do_new.reset_index(inplace=True)

In [ ]:
df_do_new

In [ ]:
df_ts3

In [ ]:
df_merged2 = pd.merge(df_ts3, df_do_new, left_index=True, right_index=True)

In [ ]:
df_merged2

In [ ]:
df_merged2.redshift.size

In [ ]:
df_merged2['diff_ra'] = 3600.*(df_merged2['ra_x'] - df_merged2['ra_y'])
df_merged2['diff_dec'] = 3600.*(df_merged2['dec'] - df_merged2['decl'])

In [ ]:
df_merged2.hist('diff_ra')

In [ ]:
df_merged2.hist('diff_dec')

In [ ]:
df_merged2.plot('redshift', 'rPSFluxMax', kind='scatter')

In [ ]:
d2d.arcsecond

In [ ]:
kd_matches = c_do[idx]
kd_matches

In [ ]:
kd_matches.separation(c_ts3)

In [ ]:
c_do[37840]

In [ ]:
c_ts3[0]

In [ ]:
3600.*abs(62.2996739-62.2996762)

In [ ]:
3600.*abs(37.1494168 - 37.1494067)

In [ ]:
len(kd_matches.separation(c_ts3))